### Heatmap of the top fGWAS results

In [21]:
setwd("../private_output/Fgwas/")

In [1]:
library(plotrix)
suppressPackageStartupMessages(library(gplots))

In [3]:
res = read.table('~/Family1070/private_output/fgwas_analysis/A_fib/pipeline_out/Final_Model/Final_Model.bfs', header=T)
seg = read.table('~/Family1070/private_output/fgwas_analysis/A_fib/pipeline_out/Final_Model/Final_Model.segbfs', header=T)
res = merge(res,seg[,c('chunk', 'PPA')], by="chunk" )
res$PPA = res$PPA.x *res$PPA.y
res = res[order(res$PPA, decreasing=T),]

*Choose a cutoff of  PPA, then re-adjust the PPA by multipling by 1 if the Locus is known

In [14]:
seg = res[res$PPA >0.30, ]

In [15]:
seg = seg[order(seg$PPA, decreasing=T),]

In [16]:
seg[seg$NKX25_ASE==1,]

,chunk,id,chr,pos,logBF,Z,V,pi,pseudologPO,pseudoPPA,PPA.x,H3K27AC,NKX25_ASE,NKX25,ATAC,H3K27AC_ASE,PPA.y,PPA
5520702,1378,rs3807989,7,116186241,26.32210,-7.67200,0.00015625,-2.06986,20.86880,1.000000,0.988999,1,1,1,1,0,1.000000,0.9889990
2040672,508,rs6801957,3,38767315,8.55653,-4.78906,0.00016384,-2.65282,2.51836,0.925419,0.882880,1,1,1,1,0,0.931087,0.8220381


In [17]:
gwas = "/publicdata/gwas_summary_stats_20180124/AF_Christophersen2017/AF_GWAS_ALLv31_maf0.01.txt"
p = paste( "grep -E '", paste(seg$id, collapse="|"), "' ", gwas ," > AF_selected_variants.tsv", sep="")

In [22]:
system(p)

In [23]:
r = read.table("AF_selected_variants.tsv")

In [25]:
seg = merge(seg, r[,c('V1', 'V8')], by.x="id", by.y="V1")
colnames(seg)[ncol(seg)] = "P_value"

In [33]:
writeLines(as.character(seg$id), "TOP_rs_variants_AF")

In [34]:
system("module load cardips")

command1 = "vcftools --vcf /publicdata/haploreg_v4.0_20151021/haploreg_v4.0_20151021.vcf --snps TOP_rs_variants_AF --recode --recode-INFO-all --out AF"
### double escape to use in R
command2 = paste("bcftools query -f ", "'%ID\\t%INFO/DBSNP\\t%INFO/NEAREST_GENCODE\\n' ",
                    "AF.recode.vcf > AF_haploreg.tab", sep="")
           
system(command1)
system(command2)

In [35]:
a = read.table("AF_haploreg.tab")
colnames(a)=c('id', 'dbSNP', 'gencode')
seg = merge(seg, a, by=1)

In [71]:
seg = seg[order(seg$PPA, decreasing=T),]

write.csv(seg, 'fgwas_results_Afib.csv', row.names=FALSE)

In [37]:
seg = subset(seg, dbSNP!="NSM")

In [62]:
se = seg[,c('NKX25_ASE','NKX25','H3K27AC','ATAC')]

In [63]:
mat = seg[rowSums(se)>0 & seg$P_value<1e-3 , c('id',  'PPA','NKX25_ASE','NKX25','H3K27AC','ATAC')]

In [64]:
mat$GWAS_P = mat$id %in% seg$id[seg$P_value<1e-8]
#mat$GWAS_other      = mat$id %in% seg$id[seg$P_value<1e-4]

mat2 = mat[,2:length(mat)]
rownames(mat2)=mat$id

lab = mat2
lab[, 2:length(mat2)]<-NA
lab[,1]<-round(lab[,1],2)

suppressPackageStartupMessages(library(RColorBrewer))
cols = brewer.pal(9, "Reds")[2:8]

breaks=seq(0.5, 1.1, by=0.1) # cutoff 0.5
breaks=seq(0.3, 1.1, by=0.1) # cutoff 0.3

my_palette = c(colorRampPalette(cols)(length(breaks)-2), "gray30")

In [65]:
mat2[mat2==0]<- NA
mat2[mat2==TRUE]<- 1
mat2[mat2==1]<- 2

In [69]:
pdf('Table_snps_AF.pdf')
heatmap.2(as.matrix(mat2[1:5,]), 
         Colv=F,Rowv=F, srtRow=0, srtCol=45, offsetRow=-1, offsetCol=-0.5,dendrogram="none",
          keysize=1, margins =c(30,26), trace="none",key.title="OR", tracecol=NA,
          cexRow=1, cexCol=1, na.color="white", col=my_palette, breaks=breaks, cellnote=lab, notecex=0.8)
dev.off()

png 
  2